# Fuzzy C Means
    Clustering algorithm testing

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from PIL import Image
from FuzzyCMeans import FuzzyCMeans

### 04 groups example
    Run clustering example against a test dataset of points visually distributed among 04 groups

In [ ]:

# Read dataset
df = pd.read_csv(filepath_or_buffer='./file/ds-fcm.csv', header=None, names=['x', 'y'])

# Determine centroids
k = 4
cMeans = FuzzyCMeans(X=df.values, k=k, m=2, maxDelta=.001, maxEpochs=1000)
centroids = cMeans.fit().getCentroids()

# Plotting parameters
radius = .5
centroidColors = ['r', 'y', 'm', 'g']

minX = np.min(centroids[:, 0])
maxX = np.max(centroids[:, 0])
minY = np.min(centroids[:, 1])
maxY = np.max(centroids[:, 1])

graphPadding = .2 + radius
xRange = (minX - graphPadding, maxX + graphPadding)
yRange = (minY - graphPadding, maxY + graphPadding)

aux = 5
sizeR = aux * (yRange[1] - yRange[0])
sizeC = aux * (xRange[1] - xRange[0])

# Plot centroids
plt.figure(figsize=(sizeR, sizeC))
ax = plt.gca()

ax.cla() # Clear (what?) things
ax.set_xlim(xRange)
ax.set_ylim(yRange)

for j in range(k):
    x, y = centroids[j, :]
    circle = plt.Circle((x, y), radius, color=centroidColors[j], fill=False)
    ax.add_patch(circle)

# Plot points
plt.scatter(df['x'].values, df['y'].values, color='w', marker='.')

## Semantic Segmentation
    TODO: ADD Description

### Utils

In [ ]:

def getImg(fileName: str, rescale: float = 1) -> Image:
    '''
        Returns an object representing the files image as a multi dimensional matrix.
    '''

    photo = Image.open(fileName).convert('RGB')
    height, width = photo.size
    imgSize = (int(height / rescale), int(width / rescale))
    return photo.resize(imgSize, Image.ANTIALIAS)

def getPixelsDataFrame(img: Image) -> pd.DataFrame:
    '''
        Returns a 02 dimensions matrix data frame of the image pixels.
        Shape: [rows]*[columns] lines X 03 columns (r, g, and b);

        - img: The image object that represents it as a multi dimensional matrix: [rows] X [columns] X [rgb channels];
    '''
    
    pixelsMultiD = np.asarray(img)
    height, width, _ = pixelsMultiD.shape
    rows = height * width

    pixels2d = np.zeros( (rows, 3) )

    for i in range(height):
        lower = i * width
        upper = lower + width

        pixels2d[lower:upper, 0] = pixelsMultiD[i, :, 0] # red
        pixels2d[lower:upper, 1] = pixelsMultiD[i, :, 1] # green
        pixels2d[lower:upper, 2] = pixelsMultiD[i, :, 2] # blue

    return pd.DataFrame(pixels2d, columns=['r', 'g', 'b'])

### Main

In [ ]:
# Set parameters
PATH_IMG_DIR = './file/image-test/'
imgConfigs = [
    { 'name': 'photo001.jpg', 'k': 10 },
]

# Open image
imgConfig = imgConfigs[0]
fileName = os.path.join(PATH_IMG_DIR, imgConfig.get('name'))
img = getImg(fileName=fileName, rescale=1)
pixelsDF = getPixelsDataFrame(img)

# Determine centroids
partition = pixelsDF.values / 255
cMeans = FuzzyCMeans(X=pixelsDF.values, k=imgConfig.get('k'), m=2, maxDelta=.02, maxEpochs=1000, partition=partition)
# centroids = cMeans.fit().getCentroids()
centroids = cMeans.getCentroids()

# Config plotting
fig = plt.figure(figsize=(20, 15))
ax = fig.add_subplot(projection='3d')

ax.azim = -45
ax.elev = 10
ax.dist = 9

ax.set_xlabel('r')
ax.set_ylabel('g')
ax.set_zlabel('b')

# Plot pixels in 3d space
ax.scatter(pixelsDF['r'].values, pixelsDF['g'].values, pixelsDF['b'].values, marker='.', color='b')

# Plot centroids in 3d space
ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], marker='o', color='r')

plt.show()